# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import sys
sys.executable


'/Users/sil/miniconda3/envs/jupyter_env/bin/python'

In [2]:
!pip install scipy


  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/c7/d9/214971dae573bd7e9303b56d2612dae439decbfc0dae0f539a591c0562ce/scipy-1.12.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 13.3 MB/s eta 0:00:0000:0100:01


In [3]:
import sys
sys.version


'3.10.13 (main, Sep 11 2023, 08:39:02) [Clang 14.0.6 ]'

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
#group by (customerID, ProductName) -->sum(quantity)
df_step1 = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': 'sum'}).reset_index()
df_step1

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
#pivot table --> df, values, index, columns, --> fill NA 0 -->fill_values 0
df_step2 = df_step1.pivot_table(index='ProductName', columns='CustomerID', values='Quantity', fill_value=0)
df_step2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
#pdist, squareform: 1ºhacer el import scipy, 2º df=pdis()--> 3ºsquareform(df)metric('euclidean')
distance = pdist(df_step2.values.T, metric='euclidean')

# Convert the distance to a squareform matrix
similarity_matrix = 1 / (1 + squareform(distance))
#similarity_matrix
df_step3 = pd.DataFrame(similarity_matrix, index=df_step2.columns, columns=df_step2.columns)
df_step3

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [8]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(df_step2, 'euclidean'))),
                                index=CustomerID.columns,
                                columns=ProductName.columns)

euclid_dist_norm

NameError: name 'CustomerID' is not defined

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [26]:
#filter df(customerID) --> ordenar(sort)-->select TOP5 (df.head())
df_step4 = df_step3.sort_values(by= ['CustomerID']).head(5)
df_step4

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [10]:
customer_id = 33

# Get the similarity scores for the target customer
similarities = df_step3[customer_id]

# Find the top 5 most similar customers
top_similar_customers = similarities.nlargest(6)[1:]
top_similar_customers

CustomerID
264     0.087047
3317    0.087047
3535    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [11]:
#df(step1)+df(step4--> customerID)}merge join} final df-->customerID...
step5 = pd.concat([df_step1, df_step4], axis=1)
step5

,CustomerID,ProductName,Quantity,33,200,264,356,412,464,477,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,33,Apricots - Dried,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33,Assorted Desserts,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33,Bandage - Flexible Neon,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33,"Bar Mix - Pina Colada, 355 Ml",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33,"Beans - Kidney, Canned",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63623,98200,Vol Au Vents,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63624,98200,Wasabi Powder,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63625,98200,Wine - Fume Blanc Fetzer,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63626,98200,Wine - Hardys Bankside Shiraz,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
registros_clientes_similares = df_step1[df_step1['CustomerID'].isin(top_similar_customers.index)]
registros_clientes_similares

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [16]:
#group by()-->sum()rank-->sort,ascending=false
total_productos_por_cliente = registros_clientes_similares.groupby('ProductName')['Quantity'].sum()
total_productos_por_cliente = total_productos_por_cliente.sort_values(ascending=False)
total_productos_por_cliente

ProductName
Butter - Unsalted                3
Wine - Ej Gallo Sierra Valley    3
Towels - Paper / Kraft           3
Soup - Campbells Bean Medley     3
Wine - Blue Nun Qualitatswein    3
                                ..
Hinge W Undercut                 1
Ice Cream Bar - Hageen Daz To    1
Jagermeister                     1
Jolt Cola - Electric Blue        1
Yogurt - French Vanilla          1
Name: Quantity, Length: 218, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [22]:
#filter(-=0); top5
# Crear un DataFrame con la clasificación de productos según su popularidad entre los 5 clientes más similares
productos_populares = df_step3.loc[:, top_similar_customers.index].sum(axis=1).sort_values(ascending=False).reset_index()
productos_populares.columns = ['ProductName', 'TotalPurchased']

productos_populares.head(5)

,ProductName,TotalPurchased
0,3535,1.359687
1,3317,1.358501
2,2503,1.352898
3,3305,1.348045
4,264,1.337637


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [25]:
#recommendations= [{},{},{}]-->for
# Paso 1: Crear un diccionario vacío para almacenar las recomendaciones
recommendations_dict = {}

# Paso 2: Crear una lista de CustomerIDs únicos
unique_customer_ids = df_step1['CustomerID'].unique()

# Pasos 3-7: Iterar sobre la lista de clientes y agregar recomendaciones al diccionario
for customer_id in unique_customer_ids:
    # Pasos 4-7: Replicar el código de los pasos anteriores para cada cliente
    target_customer_id = customer_id
    similarities = customer_similarity_df[target_customer_id]
    top_similar_customers = similarities.nlargest(6)[1:]
    similar_customers_records = df[df['CustomerID'].isin(top_similar_customers.index)]

    productos_populares = customer_product_matrix.loc[:, top_similar_customers.index].sum(axis=1).sort_values(ascending=False).reset_index()
    productos_populares.columns = ['ProductName', 'TotalPurchased']

    merged_df = pd.merge(productos_populares, customer_product_matrix, on='ProductName')
    productos_no_comprados = merged_df[merged_df[target_customer_id] == 0]
    top_productos_no_comprados = productos_no_comprados.head(5)

    # Paso 8: Agregar las recomendaciones al diccionario
    recommendations_dict[customer_id] = top_productos_no_comprados['ProductName'].tolist()




'\n# Pasos 3-7: Iterar sobre la lista de clientes y agregar recomendaciones al diccionario\nfor customer_id in unique_customer_ids:\n    # Pasos 4-7: Replicar el código de los pasos anteriores para cada cliente\n    target_customer_id = customer_id\n    similarities = customer_similarity_df[target_customer_id]\n    top_similar_customers = similarities.nlargest(6)[1:]\n    similar_customers_records = df[df[\'CustomerID\'].isin(top_similar_customers.index)]\n\n    productos_populares = customer_product_matrix.loc[:, top_similar_customers.index].sum(axis=1).sort_values(ascending=False).reset_index()\n    productos_populares.columns = [\'ProductName\', \'TotalPurchased\']\n\n    merged_df = pd.merge(productos_populares, customer_product_matrix, on=\'ProductName\')\n    productos_no_comprados = merged_df[merged_df[target_customer_id] == 0]\n    top_productos_no_comprados = productos_no_comprados.head(5)\n\n    # Paso 8: Agregar las recomendaciones al diccionario\n    recommendations_dict[cu

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.